# Шахвалиева Юлиана

### Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, log_loss
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
import time
from joblib import load
from scipy.special import logit
from scipy.stats import norm
import warnings

warnings.filterwarnings("ignore")

### Чтение входных данных

In [2]:
data = pd.read_csv('../data/data.csv')
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


Так как для задания требуется только последний день из датасета, то для минимизации памяти сразу будем хранить только его

In [3]:
# Функция для получения наблюдений за последний день
def get_last_day(data: pd.DataFrame) -> pd.DataFrame:
    # Изменение типа признака date_time
    data['date_time'] = pd.to_datetime(data['date_time'])

    # Получение наблюдений из последнего дня
    last_day = data['date_time'].dt.date.max()
    data = data[data['date_time'].dt.date >= last_day]
    
    return data

In [4]:
# Применение функции для получения наблюдений за последний день
data = get_last_day(data)

In [5]:
# Сохранение выборки для воспроизводимости
data.to_csv('../data/data_test.txt', index=False)

In [2]:
# Загрузка сохраненной выборки
data = pd.read_csv('../data/data_test.txt')
data['date_time'] = pd.to_datetime(data['date_time'])

### Анализ входных данных

In [5]:
# Функция для форматирования текста
def get_pretty_title(text):
    return ('\033[1m' + text + '\033[0m').center(55)

# Функция для форматирования вывода в консоль
def pretty_print(text, values):
    print(f'\n\n{get_pretty_title(text)}', values, sep='\n\n')

# Функция для первичного анализа входных данных
def pre_analysis(data: pd.DataFrame):  
    # Размер датасета
    shape = data.shape
    pretty_print('Размер датасета:', shape)
    
    # Просмотр типов входных данных
    dtypes = data.dtypes
    pretty_print('Типы данных:', dtypes)
    
    # Подсчет описательных статистик для всех признаков
    statistics = data.describe()
    pretty_print('Описательные статистики:', statistics)
    
    # Поиск пропусков в данных
    nan_values = data.isna().sum()
    pretty_print('Пропуски в данных:', nan_values)
    
    # Подсчет количества дубликатов
    count_duplicates = data.duplicated().sum()
    pretty_print('Количество дубликатов:', count_duplicates)
    
    # Подсчет количества случаев, когда banner_id0 не совпадает совпадает с banner_id
    count_errors = len(data[data.banner_id0 != data.banner_id])
    pretty_print('Количество несовпадений banner_id0 с banner_id:', count_errors)
    
    # Распределение целевой переменной по классам
    count_target_value = data['clicks'].value_counts()
    pretty_print('Распределение целевой переменной:', count_target_value) 

In [6]:
# Применение функции для первичного анализа входных данных
pre_analysis(data)



                Размер датасета:               

(2128978, 17)


                  Типы данных:                 

date_time          datetime64[ns]
zone_id                     int64
banner_id                   int64
oaid_hash                   int64
campaign_clicks             int64
os_id                       int64
country_id                  int64
banner_id0                  int64
rate0                     float64
g0                        float64
coeff_sum0                float64
banner_id1                  int64
rate1                     float64
g1                        float64
coeff_sum1                float64
impressions                 int64
clicks                      int64
dtype: object


            Описательные статистики:           

                           date_time       zone_id     banner_id  \
count                        2128978  2.128978e+06  2.128978e+06   
mean   2021-10-02 13:23:48.776831744  9.367876e+01  4.289836e+02   
min              2021-10-02 00:00:00 

#### Выводы из проведенного анализа: 
+ Признак `impressions` везде имеет значение `1`, поэтому его необходимо удалить;
+ Количество входнных данных: `2128978`;
+ В данных есть пропуски;
+ В данных присутствуют `1183` одинаковых записей, от них необходимо избавиться;
+ В данных присутствуют `238416` несовпадений banner_id0 с banner_id, эти ситуации необходимо удалить;
+ Целевая переменная распределена неравномерно, предсставителей одного класса сильно больше, чем представителей другого.

In [7]:
# Функция для очистки данных
def clean(data: pd.DataFrame) -> pd.DataFrame:
    # Удаление признака impressions
    data = data.drop(['impressions'], axis=1)
    
    # Удаление пропусков в данных
    data = data.dropna(axis=0, how='any')
    
    # Удаление дубликатов
    data = data.drop_duplicates()
    
    # Удаление случаев, когда banner_id0 не совпадает совпадает с banner_id
    data = data.drop(data[data.banner_id0 != data.banner_id].index)
    
    return data

In [8]:
# Применение функции для очистки данных
data = clean(data)

In [9]:
# Повтороное применение функции для первичного анализа входных данных с целью проверки результатов очистки данных
pre_analysis(data)



                Размер датасета:               

(1884528, 16)


                  Типы данных:                 

date_time          datetime64[ns]
zone_id                     int64
banner_id                   int64
oaid_hash                   int64
campaign_clicks             int64
os_id                       int64
country_id                  int64
banner_id0                  int64
rate0                     float64
g0                        float64
coeff_sum0                float64
banner_id1                  int64
rate1                     float64
g1                        float64
coeff_sum1                float64
clicks                      int64
dtype: object


            Описательные статистики:           

                        date_time       zone_id     banner_id     oaid_hash  \
count                     1884528  1.884528e+06  1.884528e+06  1.884528e+06   
mean   2021-10-02 13:11:17.356896  9.595808e+01  4.254054e+02  4.616490e+18   
min           2021-10-02 00:00:00  0.0

### Расчет clipped IPS

Формула для расчета:

$\hat{V}_{CIPS} \ (\pi, D_0) = \frac{1}{n} \sum_i {r_i \min[\frac{\pi_1(a_i|x_i)}{\pi_0(a_i|x_i)}, \lambda]}$

В данной работе: $\lambda=10$

Для расчета $\pi(a_i|x_i)$ необходимо уметь считать вероятность того, что одна нормальная случайная величина больше другой.

$\xi_1 \sim \mathcal{N}(a_1, \delta_1^2)$

$\xi_2 \sim \mathcal{N}(a_2, \delta_2^2)$

$\mathbb{P}(\xi_1 > \xi_2) = \mathbb{P}(\xi_1 - \xi_2 > 0)$

Пусть $\eta = \xi_1 - \xi_2 \rightarrow \mathbb{P}(\xi_1 - \xi_2 > 0) = \mathbb{P}(\eta > 0) = 1 - \mathbb{P}(\eta \leq 0) = 1 - \mathbb{F}_{\eta}(0)$

$\mathbb{E}(\eta) = \mathbb{E}(\xi_1 - \xi_2) = \mathbb{E}(\xi_1) - \mathbb{E}(\xi_2) = a_1 - a_2$

$\mathbb{D}(\eta) = \mathbb{D}(\xi_1 - \xi_2) = \mathbb{D}(\xi_1) + (-1)^2 \cdot \mathbb{D}(\xi_2) = \mathbb{D}(\xi_1) + \mathbb{D}(\xi_2) = \delta_1^2 + \delta_2^2$

$\eta \sim \mathcal{N}(a_1 - a_2, \delta_1^2 + \delta_2^2)$

То есть, чтобы найти вероятность того, что одна нормальная случайная величина больше другой, нужно от единицы отнять значение функции распределения с параметрами $\mathcal{N}(a_1 - a_2, \delta_1^2 + \delta_2^2)$ в точке ноль.

In [10]:
# Функция для преобразования данных перед входом в модель
def prepare_data_for_model(data: pd.DataFrame) -> pd.DataFrame: 
    # Перечень столбцов, необходимых для работы модели
    usefull_clmns = ['date_time', 'zone_id', 'banner_id', 'campaign_clicks', 'os_id', 'country_id']
    data = data[usefull_clmns]
    
    # Создание фичей на основе признака date_time: категориальный признак, отвечающий за день недели 
    # и категориальный признак, отвечающий за время суток: утро, день, вечер, ночь
    data['day'] = data['date_time'].dt.dayofweek
    data['time'] = pd.cut(data['date_time'].dt.hour, bins=[-1, 7, 12, 18, 24], labels=[0, 1, 2, 3])
    
    # One-hot encoding категориальных признаков с помощью кодировщика из HW_1
    columns_to_encode = ['zone_id', 'banner_id', 'campaign_clicks', 'os_id', 'country_id', 'day', 'time']
    
    # Загрузка кодировщика из HW_1
    encoder = load('encoder_hw_1.joblib')
    encoder.handle_unknown='ignore'
    
    X_test = data.drop(['date_time'], axis=1)
    
    # Использование кодировщика из HW_1
    X_test_encoded = encoder.transform(X_test[columns_to_encode])
    X_test = hstack([X_test.drop(columns=columns_to_encode).values, X_test_encoded])
    
    return X_test

In [11]:
# Загрузка модели из HW_1
model = load('model_hw_1.joblib')

In [18]:
lamda = 10

# Функция для расчета pi_0
def get_pi_0(df: pd.DataFrame): 
    mean = df['coeff_sum0'] - df['coeff_sum1']
    std = np.sqrt(df['g0'] ** 2 + df['g1'] ** 2) + 1e-13
    return 1 - norm.cdf(0, loc=mean, scale=std)

# Функция для расчета суммы коэффициентов
def get_coeff_sum(data: pd.DataFrame):
    X_test = prepare_data_for_model(data)
    y_pred = model.predict_proba(X_test)[:, 1]
    return logit(y_pred)

# Функция для расчета pi_0
def get_pi_1(df: pd.DataFrame): 
    mean = df['coeff_sum0_new'] - df['coeff_sum1_new']
    std = np.sqrt(df['g0'] ** 2 + df['g1'] ** 2) + 1e-13
    return 1 - norm.cdf(0, loc=mean, scale=std)

# Функция для расчета ips в каждой строчке
def get_ips(df: pd.DataFrame): 
    return df['clicks'] * min(df['pi_1'] / (df['pi_0']  + 1e-13) , lamda)\

# Обертка для запуска функций, выводяющая отладочную информацию в консоль
def count_clmn(data, clmn_name, func, k, full_dataset=False):
    print(f'[{k}/5] Counting {clmn_name}...')
    
    start = time.time()
    
    if not full_dataset:
        data[clmn_name] = data.apply(func, axis=1)
    else:
        data[clmn_name] = func(data)
    
    end = time.time()
    spent_min = round((end - start) / 60, 2)
    
    print(f'[{k}/5] End counting {clmn_name}. Time spent: {spent_min} min')
    
    return data

In [19]:
# Функция для расчета сlipped IPS
def count_ips(data: pd.DataFrame):
    # Включение таймера
    start = time.time()
    
    # Расчет pi_0
    data = count_clmn(data, 'pi_0', get_pi_0, 1) 
    
    # Расчет coeff_sum0_new
    data = count_clmn(data, 'coeff_sum0_new', get_coeff_sum, 2, True) 
    
    # Замена banner_id значением banner_id1 для расчета coeff_sum1_new
    data['banner_id'] = data['banner_id1']
    
    # Расчет coeff_sum1_new
    data = count_clmn(data, 'coeff_sum1_new', get_coeff_sum, 3, True)
    
    # Расчет pi_1
    data = count_clmn(data, 'pi_1', get_pi_1, 4)
    
    # Расчет pre_ips
    data = count_clmn(data, 'pre_ips', get_ips, 5)
    
    # Расчет clipped_ips
    clipped_ips = data['pre_ips'].mean()
    
    # Остановка таймера
    end = time.time()
    spent_time = round((end - start) / 60, 2)
    
    # Вывод расчитанной метрики
    pretty_print('Clipped IPS:', clipped_ips)
    
    # Затраченное время
    pretty_print('Time spent (min):', spent_time)  

In [20]:
count_ips(data)

[1/5] Counting pi_0...
[1/5] End counting pi_0. Time spent: 3.21 min
[2/5] Counting coeff_sum0_new...
[2/5] End counting coeff_sum0_new. Time spent: 0.04 min
[3/5] Counting coeff_sum1_new...
[3/5] End counting coeff_sum1_new. Time spent: 0.04 min
[4/5] Counting pi_1...
[4/5] End counting pi_1. Time spent: 3.21 min
[5/5] Counting pre_ips...
[5/5] End counting pre_ips. Time spent: 0.27 min


                  Clipped IPS:                 

0.07502489928351075


               Time spent (min):               

6.76
